<a href="https://colab.research.google.com/github/avaneeshp23/GenAIhackathon/blob/hackathon/genaihackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.1 MB/s eta 0:00:00


In [20]:
!git clone https://github.com/avaneeshp23/GenAIhackathon.git

fatal: destination path 'GenAIhackathon' already exists and is not an empty directory.


In [21]:
import os
os.environ['GROQ_API_KEY'] = "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"

In [22]:
from groq import Groq

In [23]:
client = Groq(api_key=os.environ['GROQ_API_KEY'])

In [24]:
import pandas as pd
import requests

In [25]:
train_df = pd.read_csv('/content/GenAIhackathon/data/train.csv')

In [26]:
train_df.head()

,ticket_ID,ticket_subject,ticket_body,department,type,priority,language
0,1001,Discrepancia de facturación en Google Workspace,"Estimado equipo de soporte de TI,\n\nEstoy esc...",Billing and Payments,Incident,low,es
1,1002,Urgent Consultation Request for Critical IT Is...,"Dear IT Services Support Team, I hope this mes...",Customer Service,Request,high,en
2,1003,Consulta sobre Servicios de Consultoría en TI,"Estimado Servicio de Atención al Cliente,\n\nM...",General Inquiry,Request,medium,es
3,1004,Demande de mise à jour des dossiers,"Cher service client, \n\nJe vous écris pour de...",Human Resources,Change,low,fr
4,1005,Issues with Slack connection affecting team co...,"Dear Customer Support Team,\n\nI am encounteri...",Product Support,Problem,medium,en


In [27]:
train_df['text'] = train_df['ticket_subject'] + ' ' + train_df['ticket_body']
train_df = train_df[['text', 'department', 'type', 'priority', 'language']]

In [28]:
train_df.head()

,text,department,type,priority,language
0,Discrepancia de facturación en Google Workspac...,Billing and Payments,Incident,low,es
1,Urgent Consultation Request for Critical IT Is...,Customer Service,Request,high,en
2,Consulta sobre Servicios de Consultoría en TI ...,General Inquiry,Request,medium,es
3,Demande de mise à jour des dossiers Cher servi...,Human Resources,Change,low,fr
4,Issues with Slack connection affecting team co...,Product Support,Problem,medium,en


In [29]:
import os
api_key = os.getenv('GROQ_API_KEY')
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

In [30]:
data = {
    'model': 'deepseek-r1-distill-llama-70b',
    'train_data': train_df.to_dict(orient='records'),
    'task': 'classification',
    'labels': {
        'department': train_df['department'].unique().tolist(),
        'type': train_df['type'].unique().tolist(),
        'priority': train_df['priority'].unique().tolist(),
        'language': train_df['language'].unique().tolist()
    }
}

In [31]:
#Training the model
response = requests.post('https://api.groq.com/v1/chat/completions', headers=headers, json=data)
if response.status_code == 200:
    print("Training started successfully!")
    print(response.json())
else:
    print("Error:", response.status_code, response.text)

Error: 404 {"error":{"message":"Unknown request URL: POST /v1/chat/completions. Please check the URL for typos, or see the docs at https://console.groq.com/docs/","type":"invalid_request_error","code":"unknown_url"}}



In [32]:
completion = client.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",
    messages=[
        {
            "role": "user",
            "content": "You are given a pandas dataframe train_df. You task is to automatically analyze and classify IT support tickets based on their content. Each ticket, consisting of a subject and body, should be accurately classified to predict the following attributes:\n\nDepartment: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry\nPriority: The urgency level such as low, medium, or high.\nLanguage: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following: https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html.\nType: The nature of the request, categorized as Incident, Request, Problem, or Change."
        }
    ],
    temperature=0.6,
    max_completion_tokens=4096,
    top_p=0.95,
    stream=True,
    stop=None,
)

In [34]:
def get_response(message, model="deepseek-r1-distill-llama-70b"):
    task_description = (
        "You are given a pandas dataframe row from train_df. Your task is to automatically analyze and classify IT support tickets based on their content. "
        "Each ticket, consisting of a Text, should be accurately classified to predict the following attributes:\n\n"
        "Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, "
        "Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry\n"
        "Priority: The urgency level such as low, medium, or high.\n"
        "Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following: "
        "https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html.\n"
        "Type: The nature of the request, categorized as Incident, Request, Problem, or Change."
    )
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": task_description},
            {"role": "user", "content": message}
        ],
        temperature=0.6,
        max_completion_tokens=4096,
        top_p=0.95,
        stream=False,
        stop=None,
    )
    return response.choices[0].message.content

In [35]:
results = []
for _, row in train_df.iterrows():
    message = row['text']
    response = get_response(message)
    results.append(response)

train_df['classification'] = results

In [36]:
train_df.head()

,text,department,type,priority,language,classification
0,Discrepancia de facturación en Google Workspac...,Billing and Payments,Incident,low,es,"<think>\nOkay, so I need to classify this IT s..."
1,Urgent Consultation Request for Critical IT Is...,Customer Service,Request,high,en,"<think>\nOkay, so I'm trying to figure out how..."
2,Consulta sobre Servicios de Consultoría en TI ...,General Inquiry,Request,medium,es,"<think>\nOkay, I need to classify this IT supp..."
3,Demande de mise à jour des dossiers Cher servi...,Human Resources,Change,low,fr,"<think>\nOkay, I'm trying to figure out how to..."
4,Issues with Slack connection affecting team co...,Product Support,Problem,medium,en,"<think>\nOkay, so I need to classify this IT s..."


In [38]:
test_df = pd.read_csv('/content/GenAIhackathon/data/test.csv')
test_df.head()

,ticket_ID,ticket_subject,ticket_body
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...
